# Beijing Air-Quality Time Series Project
### Data Cleaning Notebook

by Dolci Sanders and Paul Torres


### Import Libraries

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import glob
from sklearn.model_selection import train_test_split
import pickle

### Read in Data

Our Data is provided by the UCI Machine Learning Repository
Beijing Multi-Site Air-Quality Data

Once Read in, we will look at the head and convert our time into data time. 

We have 12 data sets, one from each reporting site, to concatenate together to get the whole picture of Beijing's Air Quality. 


In [2]:
time = pd.read_csv('DATA/PRSA_Data_Tiantan_20130301-20170228.csv', index_col = None)

In [3]:
time.year.unique()

array([2013, 2014, 2015, 2016, 2017])

### Date Time Formatting
Crucial to time series. 

In [4]:
time['Date'] = pd.to_datetime(time[['year','month','day','hour']])
time = time.set_index('Date')

In [5]:
time.shape

(35064, 18)

### Dealing with Nan Values
After looking at the data, we wanted to look at null values and figure out what to do with these. 

We first calculated the mean and media for all of the values. These values varied wildly and we were concerned they would have a negative effect on the predictions. 

However, upon further research, other methods have proven more effective in inputing the data for time series such as Interpolating Time, however because we have hourly time this was not the best method. We elected to try imputing using the InterpolateLinear method. 

In [6]:
time.isna().sum()

No            0
year          0
month         0
day           0
hour          0
PM2.5       677
PM10        597
SO2        1118
NO2         744
CO         1126
O3          843
TEMP         20
PRES         20
DEWP         20
RAIN         20
wd           78
WSPM         14
station       0
dtype: int64

In [7]:
time = time.interpolate(method='time')

### PM2.5 (TARGET VARIABLE)

In [8]:
print('total PM2.5 Missing Values: ', time['PM2.5'].isna().sum())
print('mean: ', time['PM2.5'].mean())
print('median: ', time['PM2.5'].median())

total PM2.5 Missing Values:  0
mean:  82.03309662331738
median:  58.0


### PM10

In [9]:
print('total PM10 Missing Values: ', time['PM10'].isna().sum())
print('mean: ', time['PM10'].mean())
print('median: ', time['PM10'].median())

total PM10 Missing Values:  0
mean:  106.53707648870636
median:  85.0


### SO2

In [10]:
print('total SO2 Missing Values: ', time['SO2'].isna().sum())
print('mean: ', time['SO2'].mean())
print('median: ', time['SO2'].median())

total SO2 Missing Values:  0
mean:  14.510017738991555
median:  7.0


### NO2

In [11]:
print('total NO2 Missing Values: ', time['NO2'].isna().sum())
print('mean: ', time['NO2'].mean())
print('median: ', time['NO2'].median())

total NO2 Missing Values:  0
mean:  53.25882833532967
median:  47.0


### CO

In [12]:
print('total CO Missing Values: ', time['CO'].isna().sum())
print('mean: ', time['CO'].mean())
print('median: ', time['CO'].median())

total CO Missing Values:  0
mean:  1305.3332620351357
median:  900.0


### O3

In [13]:
print('total O3 Missing Values: ', time['O3'].isna().sum())
print('mean: ', time['O3'].mean())
print('median: ', time['O3'].median())

total O3 Missing Values:  0
mean:  56.14807717887289
median:  40.0


### TEMP

In [14]:
print('total TEMP Missing Values: ', time['TEMP'].isna().sum())
print('mean: ', time['TEMP'].mean())
print('median: ', time['TEMP'].median())

total TEMP Missing Values:  0
mean:  13.668249517775271
median:  14.6


### PRES

In [15]:
print('total PRES Missing Values: ', time['PRES'].isna().sum())
print('mean: ', time['PRES'].mean())
print('median: ', time['PRES'].median())

total PRES Missing Values:  0
mean:  1012.5518711023906
median:  1012.2


### DEWP

In [16]:
print('total DEWP Missing Values: ', time['DEWP'].isna().sum())
print('mean: ', time['DEWP'].mean())
print('median: ', time['DEWP'].median())

total DEWP Missing Values:  0
mean:  2.4451260552133287
median:  3.0


### RAIN

In [17]:
print('total RAIN Missing Values: ', time['RAIN'].isna().sum())
print('mean: ', time['RAIN'].mean())
print('median: ', time['RAIN'].median())

total RAIN Missing Values:  0
mean:  0.06398300250969628
median:  0.0


### wd

wd is wind direction and is not an integer. We will forward fill the missing values. As wind direction does not change rapidly. --A few of these (4) did not change from nan's and for those stuborn nan values, we used back fill for these. 

In [18]:
time.wd = time.wd.fillna(method = 'ffill')
time.wd = time.wd.fillna(method = 'bfill')

print('total wd Missing Values: ', time['wd'].isna().sum())


total wd Missing Values:  0


### WSPM

In [19]:
print('total WSPM Missing Values: ', time['WSPM'].isna().sum())
print('mean: ', time['WSPM'].mean())
print('median: ', time['WSPM'].median())

total WSPM Missing Values:  0
mean:  1.8608145106091456
median:  1.5


### Nan Values Filled, check. 

In [20]:
time.isna().sum()

No         0
year       0
month      0
day        0
hour       0
PM2.5      0
PM10       0
SO2        0
NO2        0
CO         0
O3         0
TEMP       0
PRES       0
DEWP       0
RAIN       0
wd         0
WSPM       0
station    0
dtype: int64

## Combine 12 Testing Site Tables
We have 12 testing sites with a table for each. 
So we built a function to automate our concatenation of all of these. 
For each test site, we loop through these, creating a whole dataframe for visualizations and also a separate loop of this to train test split, interpolate the test then the train using the time method, finally we back fill the stubborn few.

### Time DataFrame, import all as a whole, interpolate by table

In [8]:
# From the DATA folder, we wil run through all of the individual csv files. 

path = r'DATA/' 
allFiles = glob.glob(path + '/*.csv')

# Prep the test and train data frames. 
time = pd.DataFrame()
times = []


for file_ in allFiles:
    
    # Read in and set data index to df
    df = pd.read_csv(file_,index_col = None,header = 0)
    df['Date'] = pd.to_datetime(df[['year','month','day','hour']])
    df = df.set_index('Date')
    df = df.loc[:'2016-12-31 23:00:00']
    
    # Drop unwanted columns from df into the new data frame time
    
    time_ = df.drop(columns=['No'], axis = 1)
    time_.columns = ['year','month','day','hour','PM2.5','PM10', 'SO2', 'NO2', 'CO', 'O3', 'TEMP', 'PRES', 'DEWP', 'RAIN','wd', 'WSPM', 'station']
    
    
    # Interpolate missing values in time, append times list with each iteration
    
    time_ = time_.interpolate(method = 'time')
    time_.wd = time_.wd.fillna(method = 'ffill')
    time_.NO2 = time_.NO2.fillna(method = 'bfill')
    time_.O3 = time_.O3.fillna(method = 'bfill')
    time_.CO = time_.CO.fillna(method = 'bfill')
    times.append(time_)
    
# Make times into the time df, this is the entire df of all 12 tables for visualization purposes
time = pd.concat(times)
time = time.sort_values(['station', 'Date'])

    

In [9]:
time.head()

,year,month,day,hour,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,wd,WSPM,station
Date,,,,,,,,,,,,,,,,,
2013-03-01 00:00:00,2013,3,1,0,4.0,4.0,4.0,7.0,300.0,77.0,-0.7,1023.0,-18.8,0.0,NNW,4.4,Aotizhongxin
2013-03-01 01:00:00,2013,3,1,1,8.0,8.0,4.0,7.0,300.0,77.0,-1.1,1023.2,-18.2,0.0,N,4.7,Aotizhongxin
2013-03-01 02:00:00,2013,3,1,2,7.0,7.0,5.0,10.0,300.0,73.0,-1.1,1023.5,-18.2,0.0,NNW,5.6,Aotizhongxin
2013-03-01 03:00:00,2013,3,1,3,6.0,6.0,11.0,11.0,300.0,72.0,-1.4,1024.5,-19.4,0.0,NW,3.1,Aotizhongxin
2013-03-01 04:00:00,2013,3,1,4,3.0,3.0,12.0,12.0,300.0,72.0,-2.0,1025.2,-19.5,0.0,N,2.0,Aotizhongxin


### Train Test Split Interpolation with 80-20 Split

In [10]:
# From the DATA folder, we wil run through all of the individual csv files. 

path = r'DATA/' 

allFiles = glob.glob(path + '/*.csv')

# Prep the test and train data frames.

test = pd.DataFrame()
train = pd.DataFrame()
trains = []
tests = []



for file_ in allFiles:
    
    # Read in and set data index. 
    df = pd.read_csv(file_,index_col = None,header = 0)
    df['Date'] = pd.to_datetime(df[['year','month','day','hour']])
    df = df.set_index('Date')
    df = df.loc[:'2016-12-31 23:00:00']
    
    # Drop unwanted columns 
    X = df.drop(columns=['PM2.5','year','month','day','hour','No'], axis = 1)
    
    # Set the target 
    y = df['PM2.5']
    
    # Train test split to prevent data leakage
    X_train = X[:int(X.shape[0]*0.8)]
    X_test = X[int(X.shape[0]*0.8):]
    y_train = y[:int(X.shape[0]*0.8)]
    y_test = y[int(X.shape[0]*0.8):]
    
    
    train = pd.concat([X_train,y_train], axis = 1, ignore_index=True)
    train.columns = ['PM10', 'SO2', 'NO2', 'CO', 'O3', 'TEMP', 'PRES', 'DEWP', 'RAIN','wd', 'WSPM', 'station','PM2.5']
    
    test = pd.concat([X_test,y_test], axis = 1, ignore_index=True)
    test.columns = ['PM10', 'SO2', 'NO2', 'CO', 'O3', 'TEMP', 'PRES', 'DEWP', 'RAIN','wd', 'WSPM', 'station','PM2.5']
    
    # Interpolate the train data and add to the rest of the train list 
    
    train = train.interpolate(method = 'time')
    train.wd = train.wd.fillna(method = 'ffill')
    train.NO2 = train.NO2.fillna(method = 'bfill')
    trains.append(train)
    
    # Interpolate the test data and add to the rest of the test list
    
    test = test.interpolate(method = 'time')
    test['PM2.5'] = test['PM2.5'].fillna(method = 'bfill')
    test.wd = test.wd.fillna(method = 'ffill')
    test.O3 = test.O3.fillna(method = 'bfill')
    test.CO = test.CO.fillna(method = 'bfill')
    tests.append(test)

# Take the data fame we made at the top and combine the new data frame we processed here 
    
train = pd.concat(trains)
train = train.sort_values(['station', 'Date'])
test = pd.concat(tests)
test = test.sort_values(['station', 'Date'])

### Latitude and Longitude 
We will add Latidude and Longitude for each testing site for more visualizations. 


In [24]:
# Define a function that gets the coordinates of the cities

from geopy.geocoders import Nominatim

def get_lat_long(city):
    country = 'China'
    geolocator = Nominatim(user_agent = 'Beijing')
    location = geolocator.geocode(city+','+ country)
    long = location.longitude
    lat = location.latitude
    return(lat, long)

In [25]:
# Stations are a little off

# Aotizhongxin is Aoti Zhongxin
# Wanshouxigong is Wan Shou Xin Gong

city = ['Aoti Zhongxin','Changping', 'Dingling', 'Dongsi', 'Guanyuan','Gucheng', 
        'Huairou', 'Nongzhanguan','Shunyi', 'Tiantan','Wanliu', 'Wan Shou Xin Gong']

# Use the function to get a list of the coordinates
coordinates = []
latitude = []
longitude = []

for i in city: 
    (get_lat_long(i))
    coordinates.append(get_lat_long(i))
    
for i in range(0,12): 
    latitude.append(coordinates[i][0])
    longitude.append(coordinates[i][1])


In [26]:
time['station_latitude'] = time.station
time['station_longitude'] = time.station

In [27]:
cond = [[time.station == 'Aotizhongxin'], [time.station == 'Changping'], [time.station == 'Dingling'], 
        [time.station == 'Dongsi'], [time.station == 'Guanyuan'], [time.station == 'Gucheng'],
        [time.station == 'Huairou'], [time.station == 'Nongzhanguan'], [time.station == 'Shunyi'], 
        [time.station == 'Tiantan'], [time.station == 'Wanliu'], [time.station == 'Wanshouxigong']]



time['station_latitude'] = np.select(cond, latitude)
time['station_longitude'] = np.select(cond, longitude)

ValueError: Length of values does not match length of index

In [28]:
time.columns


Index(['year', 'month', 'day', 'hour', 'PM2.5', 'PM10', 'SO2', 'NO2', 'CO',
       'O3', 'TEMP', 'PRES', 'DEWP', 'RAIN', 'wd', 'WSPM', 'station',
       'station_latitude', 'station_longitude'],
      dtype='object')

### Pickle the Time DF and the Train and Test DFs, then proceed to the EDA Notebook
The Time DF will be used for visualizations
The Train Test Split will be used in Predictions as well as for comparative visuals later on. 

In [11]:
test.to_pickle('PKL/test.pkl')
train.to_pickle('PKL/train.pkl')
time.to_pickle('PKL/time.pkl')